##### Copyright 2024 Google LLC. Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License

# Data Preparation

### Step 1: Download a csv file of the embeddings using this [link](https://docs.google.com/forms/d/e/1FAIpQLSeZLIqTCIx1-OiBzUnqXZpu_k5M223ZvMmqwQhMZ_0TkaWhEQ/viewform).

The county and ZCTA (zipcode census tabulation area) embeddings are available in different files.

Here we assume that you have obtained the embeddings and uploaded them to a Google Drive directory called `pdfm_embeddings/v0/us`.

In [ ]:
import pandas as pd
from google.colab import drive
#@markdown Specify the path to the embeddings file.
embeddings_file_path = '/content/drive/MyDrive/pdfm_embeddings/v0/us/county_embeddings.csv' #@param {type:"string"}

drive.mount('/content/drive', force_remount=True)
county_embeddings = pd.read_csv(embeddings_file_path).set_index('place')

In [ ]:
embedding_features = [f'feature{x}' for x in range(330)]
county_embeddings.head(2)

### Step 2: Download and prepare monthly unemployment data at county level from Data Commons

In [ ]:
!pip install datacommons_pandas --upgrade --quiet
import datacommons_pandas as dc

In [ ]:
label = 'UnemploymentRate_Person'
# Due to response size limits, we'll query in batches.
batch_size = 200

all_labels = []
for start in range(0, county_embeddings.index.shape[0], batch_size):
    batch_indices = county_embeddings.index[start : start + batch_size]
    batch_data = dc.build_time_series_dataframe(batch_indices, label)
    all_labels.append(batch_data)

df_labels = pd.concat(all_labels)

print(df_labels.shape)
df_labels.head(2)

In [ ]:
_ = df_labels.loc['geoId/01001'].plot(figsize=(5, 3))

#Data Visualization

## Download the county level geojson file.

[link text](https://) The county level geojson file are available in the same folder as the embeddings. Download the geojson file into a local folder or a folder under Google drive. Here we assume that you have downloaded the file in Google Drive folder called pdfm_embeddings/v0/us

In [ ]:
import geopandas as gpd

#@markdown Specify the path to the county geojson file.
county_geojson_file_path = '/content/drive/MyDrive/pdfm_embeddings/v0/us/county.geojson' #@param {type:"string"}
geo = gpd.read_file(county_geojson_file_path).set_index('place')
embeddings = gpd.GeoDataFrame(county_embeddings, geometry=geo.geometry)
embeddings.shape

## Map out an embedding dimension spatially.

In this example, we have mapped out feature0 dimension of the embeddings data for counties in New York.

In [ ]:
def get_locale(df, index, states=None, counties=None):
  df = df[df.index.isin(index)]
  if not states and not counties:
    return df
  filter = df.state.isin(states)
  if counties:
    filter &= df.county.isin(counties)
  return df[filter]

feature = embedding_features[0]
ax = get_locale(embeddings, county_embeddings.index).plot(feature)
_ = ax.set_title(feature + ' in counties')

ax = get_locale(embeddings, county_embeddings.index, states=['NY']).plot(feature)
_ = ax.set_title(feature + ' in NY counties')

# Applying the embeddings to a Nowcasting usecase.

Assume that we have unemployment data from 2024-07 from all counties, and only a subset of counties from 2024-08, we can predict the values for the rest of the counties using the embeddings.

We have demonstrated the Nowcasting example using a LightGBM (Light Gradient Boosting Machine) model.

In [ ]:
def evaluate_model(y_true, y_pred):
    """Calculates Mean Absolute Error (MAE) and Mean Absolute Percentage Error (MAPE)."""
    return {
        'MAE': metrics.mean_absolute_error(y_true, y_pred),
        'MAPE': metrics.mean_absolute_percentage_error(y_true, y_pred),
    }

def train_and_evaluate(train_data, test_data, feature_sets, test_date, model_params=None):
    """
    Trains and evaluates models on given feature sets.

    Args:
    - train_data (pd.DataFrame): Training dataset with features and labels.
    - test_data (pd.DataFrame): Testing dataset with features and labels.
    - feature_sets (dict): Dictionary of feature set names and column lists.
    - test_date (str): Target date for prediction in test data.
    - model_params (dict, optional): Parameters for the LightGBM model.

    Returns:
    - pd.DataFrame: DataFrame with evaluation results for each feature set.
    """
    # Set default model parameters if none are provided
    if model_params is None:
        model_params = {
            'min_child_samples': 10,
            'num_leaves': 23,
            'max_bin': 100,
            'n_estimators': 100,
            'learning_rate': 0.1,
            'force_col_wise': True,
            'verbose': -1,
        }

    results = []
    for set_name, features in feature_sets.items():
        # Initialize and train the model
        model = lgbm.LGBMRegressor(**model_params)
        model.fit(train_data[features], train_data[test_date])

        # Make predictions and evaluate the model
        predictions = model.predict(test_data[features])
        eval_results = evaluate_model(test_data[test_date], predictions)
        eval_results['Feature Set'] = set_name  # Add feature set name for reference
        results.append(eval_results)

    return pd.DataFrame(results)

In [ ]:
#@title Training on 6 previous data points and embeddings
import lightgbm as lgbm
import numpy as np
from sklearn import linear_model
from sklearn import metrics

TEST_DATE = '2024-08'
TRAIN_START_DATE = '2024-02'
TRAIN_END_DATE = '2024-07'

train_dates = [x.strftime('%Y-%m') for x in pd.date_range(
    start=TRAIN_START_DATE, end=TRAIN_END_DATE, freq='MS')]

# Join embeddings with ground truth data (unemployment rate) for modeling
data = county_embeddings.join(df_labels[train_dates + [TEST_DATE]])

train_data = data.sample(n=1000, random_state=42)
test_data = data.drop(train_data.index)
print(f"Training dates: {train_dates}")
print(f"Test date: {TEST_DATE}")
print(f"# Training counties: {train_data.shape[0]}")
print(f"# Test counties: {test_data.shape[0]}")

# Define feature sets for different model configurations
# 1. Using only past unemployment data
# 2. Using only embeddings data
# 3. Combining embeddings and past unemployment data to see additive improvements
FEATURE_SETS = {
    'Past Unemployment': train_dates,
    'Embeddings Only': embedding_features,
    'Embeddings + Past Unemployment': embedding_features + train_dates,
}

MODEL_PARAMS = {
    'min_child_samples': 10,
    'num_leaves': 23,
    'max_bin': 100,
    'n_estimators': 100,
    'learning_rate': 0.1,
    'force_col_wise': True,
    'verbose': -1,
}

# Run training and evaluation for each feature set and display results
evaluation_results = train_and_evaluate(
    train_data=train_data,
    test_data=test_data,
    feature_sets=FEATURE_SETS,
    test_date=TEST_DATE,
    model_params=MODEL_PARAMS
)

print("Model Performance Results:")
evaluation_results.round(3)

The results show that PDFM embeddings improve nowcasting accuracy when used in conjunction with past data.

In [ ]:
#@title The improvements are bigger if there are fewer historical data points.
# Define new training configuration for limited historical data
MINHIST_TEST_DATE = '2024-08'
MINHIST_TRAIN_START_DATE = '2024-07'
MINHIST_TRAIN_END_DATE = '2024-07'

minhist_train_dates = [x.strftime('%Y-%m') for x in pd.date_range(
    start=MINHIST_TRAIN_START_DATE, end=MINHIST_TRAIN_END_DATE, freq='MS')]

minhist_data = embeddings.join(df_labels[minhist_train_dates + [MINHIST_TEST_DATE]])

minhist_train_data = minhist_data.sample(n=1000, random_state=42)
minhist_test_data = minhist_data.drop(minhist_train_data.index)
print(f"Training dates with minimized history: {minhist_train_dates}")
print(f"Test date: {MINHIST_TEST_DATE}")
print(f"# Training counties: {minhist_train_data.shape[0]}")
print(f"# Test counties: {minhist_test_data.shape[0]}")

MINHIST_FEATURE_SETS = {
    'Past Unemployment (MinHist)': minhist_train_dates,
    'Embeddings Only': embedding_features,
    'Embeddings + Past Unemployment (MinHist)': embedding_features + minhist_train_dates,
}

minhist_results = train_and_evaluate(
    train_data=minhist_train_data,
    test_data=minhist_test_data,
    feature_sets=MINHIST_FEATURE_SETS,
    test_date=MINHIST_TEST_DATE,
    model_params=MODEL_PARAMS  # Reuse parameters from previous model configuration
)

print("Performance with Minimized Historical Data Points:")
minhist_results.round(3)